# Advanced Querying Mongo

Importing libraries and setting up connection

In [22]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING
import re

dbName = "datamad0320"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/datamad0320


### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [49]:
list(db.companies.find({"name":{"$eq":"Babelgum"}},{"name":1, "_id":0}))


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [48]:
list(db.companies.find({"number_of_employees":{"$gt":500}},{"number_of_employees":1}).sort([
('number_of_employees', DESCENDING)
]).limit(20))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'), 'number_of_employees': 405000},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'), 'number_of_employees': 388000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'), 'number_of_employees': 320000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'), 'number_of_employees': 300000},
 {'_id': ObjectId('52cdef7e4bab8bd67529b0fe'), 'number_of_employees': 227000},
 {'_id': ObjectId('52cdef7d4bab8bd675298aa4'), 'number_of_employees': 221726},
 {'_id': ObjectId('52cdef7d4bab8bd675298b99'), 'number_of_employees': 205000},
 {'_id': ObjectId('52cdef7e4bab8bd67529a657'), 'number_of_employees': 200300},
 {'_id': ObjectId('52cdef7e4bab8bd67529aa51'), 'number_of_employees': 200000},
 {'_id': ObjectId('52cdef7d4bab8bd675299156'), 'number_of_employees': 186000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e6f'), 'number_of_employees': 180500},
 {'_id': ObjectId('52cdef7c4bab8bd675298517'), 'number_of_employees': 177000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d31'), 'numb

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [125]:
lst3 = list(db.companies.find({"founded_year": { "$in":[2000,2001,2002,2003,2004,2005]}},
{"name":1,"founded_year":1,"_id":0}))
lst3[:5]

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [124]:
lst4 = list(db.companies.find( { "$and": [ { "ipo.valuation_amount": {"$gt":100000000} }, { "founded_year": { "$lt": 2010 }}]}, 
{"name":1,"ipo":1,"_id":0} ))

lst4[:3]

[{'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [123]:
list(db.companies.find({ "$and": [ { "number_of_employees": {"$lt":1000} },{ "founded_year": { "$lt": 2005 }}]}, 
{"name":1,"number_of_employees":1,"founded_year":1,"_id":0}).sort([
('number_of_employees', DESCENDING)
]).limit(10))


[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Yelp', 'number_of_employees': 800, 'founded_year': 2004},
 {'name': 'ZoomInfo', 'number_of_employees': 800, 'founded_year': 2000},
 {'name': 'MySpace', 'number_of_employees': 800, 'founded_year': 2003}]

### 6. All the companies that don't include the `partners` field.

In [128]:
list(db.companies.find({"partners": { "$type": "null"}},{"name":1,"_id":0}))

#Creo que esto esta mal

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [137]:
lst7 = list(db.companies.find({"category_code": { "$type": "null"}},{"name":1,"category_code":1, "_id":0}))


lst7[:5]

[{'name': 'Collective', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [135]:
lst8 = list(db.companies.find({ "$and": [ { "number_of_employees": {"$gte":100} },{"number_of_employees": {"$lt": 1000}}]}, 
{"name":1,"number_of_employees":1,"_id":0}))

lst8[:5]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [143]:
list9 = list(db.companies.find({ "ipo.valuation_amount": {"$type": "int"}}, {"name":1,"ipo.valuation_amount":1,"_id":0}).sort
('ipo.valuation_amount', DESCENDING))

list9[:5]

[{'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'Chegg', 'ipo': {'valuation_amount': 1100000000}},
 {'name': 'Chegg', 'ipo': {'valuation_amount': 1100000000}},
 {'name': 'RPX Corporation', 'ipo': {'valuation_amount': 1096000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [145]:
list(db.companies.find({"number_of_employees":{"$gt":10}},{"name":1,"number_of_employees":1,"_id":0}).sort([
('number_of_employees', DESCENDING)
]).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [153]:
lst10 = list(db.companies.find({"founded_month":{"$gte":6}},{"name":1,"founded_month":1,"_id":0}).limit(1000))

lst10[:5]

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'Postini', 'founded_month': 6},
 {'name': 'Geni', 'founded_month': 6}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [196]:
lst12 = list(db.companies.find({ "$and": [ { "acquisition.price_amount": {"$gt":1000000} },{ "founded_year": { "$lt": 2000 }}]}, 
{"name":1,"acquisition.price_amount":1,"founded_year":1,"_id":0}))

lst12[:5]

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld',
  'founded_year': 1999,
  'acquisition': {'price_amount': 7140000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [198]:
lst13 = list(db.companies.find({"$and":[{"acquisition.acquired_year":{"$gt": 2010}},{"acquisition.price_amount": {"$gt":0}}]}, 
{"name":1,"acquisition":1,"_id":0}))


lst13[:2]

[{'name': 'Wetpaint',
  'acquisition': {'price_amount': 30000000,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': 'http://allthingsd.com/20131216/viggle-tries-to-bulk-up-its-social-tv-business-by-buying-wetpaint/?mod=atdtweet',
   'source_description': ' Viggle Tries to Bulk Up Its Social TV Business by Buying Wetpaint',
   'acquired_year': 2013,
   'acquired_month': 12,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Viggle', 'permalink': 'viggle'}}},
 {'name': 'Digg',
  'acquisition': {'price_amount': 500000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/07/12/betaworks-acquires-digg/',
   'source_description': 'Betaworks Acquires Digg (TechCrunch)',
   'acquired_year': 2012,
   'acquired_month': 7,
   'acquired_day': 12,
   'acquiring_company': {'name': 'betaworks', 'permalink': 'betaworks'}}}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [183]:
list14 = list(db.companies.find({ "founded_year": {"$type": "int"}}, {"name":1,"founded_year":1,"_id":0}).sort([
('founded_year', ASCENDING)
]).limit(100))

list14[:5]

[{'name': 'Alstrasoft', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'US Army', 'founded_year': 1800},
 {'name': 'DuPont', 'founded_year': 1802},
 {'name': 'McKesson', 'founded_year': 1833}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [199]:
list(db.companies.find({"$and":[{"founded_day":{"$lte": 7}},{"acquisition.price_amount": {"$gt":0}}]}, 
{"name":1,"founded_day":1,"acquisition.price_amount":1,"_id":0}).sort([
('acquisition.price_amount', DESCENDING)
]).limit(10))


[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense',
  'founded_day': 1,
  'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee',
  'founded_day': 1,
  'acquisition': {'price_amount': 321000000}},
 {'name': 'BlueLithium',
  'founded_day': 1,
  'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [200]:
lst16 = list(db.companies.find({"$and":[{"category":{"$eq": "web"}},{"number_of_employees": {"$gt":4000}}]}, 
{"name":1,"category":1,"number_of_employees":1,"_id":0}).sort('number_of_employees', ASCENDING))

lst12[:5]


[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld',
  'founded_year': 1999,
  'acquisition': {'price_amount': 7140000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [202]:
list(db.companies.find({"$and":[{"acquisition.price_amount":{"$gt":10000000}},
{"acquisition.price_currency_code": {"$eq":"EUR"}}]}, 
{"name":1,"acquisition.price_amount":1,"acquisition.price_currency_code":1,"_id":0}))



[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [203]:
lst18 = list(db.companies.find({"acquisition.acquired_month":{"$lte":3}},{"name":1,"acquisition":1,"_id":0}).limit(10))

lst18[:3]

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [209]:
lst19 = list(db.companies.find({"$and":[{"founded_year":{"$gte":2000}},
{"founded_year":{"$lte":2010}}, {"acquisition.acquired_year":{"$gt":2011}}]}, 
{"name":1, "founded_year":1, "acquisition.acquired_year":1, "_id":0}))

lst19[:5]

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'blogTV',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Revision3',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2012}}]

### 20. All the companies that have been 'deadpooled' after the third year.